##### Copyright 2019 The TensorFlow Authors.

In [114]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 정형 데이터 다루기

## 텐서플로와 필요한 라이브러리 임포트하기

In [115]:
import numpy as np
import pandas as pd

import tensorflow as tf

from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

## 판다스로 데이터프레임 만들기

[판다스](https://pandas.pydata.org/)는 정형 데이터를 읽고 조작하는데 유용한 유틸리티 함수를 많이 제공하는 파이썬 라이브러리입니다. 판다스를 이용해 URL로부터 데이터를 다운로드하여 읽은 다음 데이터프레임으로 변환하겠습니다.

In [116]:
train_df = pd.read_csv('./train.csv')
test_df = pd.read_csv('./test.csv')

In [117]:
train_x = train_df.drop(columns=['PRODUCT_ID', 'Y_Quality', 'TIMESTAMP'])
train_y = train_df['Y_Class']

test_x = test_df.drop(columns=['PRODUCT_ID', 'TIMESTAMP'])

In [118]:
from sklearn.preprocessing import LabelEncoder

qual_col = ['LINE','PRODUCT_CODE']

for i in qual_col:
    le = LabelEncoder()
    le.fit(train_x[i])
    train_x[i] = le.transform(train_x[i])

    for label in np.unique(test_x[i]):
        if label not in le.classes_:
            le.classes_ = np.append(le.classes_, label)
    
    test_x[i] = le.transform(test_x[i])

print('done')

done


In [119]:
## PRODUCT_CODE

# train
train_x_1 = train_x[train_x['PRODUCT_CODE'] == 0].drop('Y_Class', axis=1)
train_x_2 = train_x[train_x['PRODUCT_CODE'] == 1].drop('Y_Class', axis=1)
train_x_3 = train_x[train_x['PRODUCT_CODE'] == 2].drop('Y_Class', axis=1)

train_y_1 = train_x['Y_Class'][train_x['PRODUCT_CODE'] == 0]
train_y_2 = train_x['Y_Class'][train_x['PRODUCT_CODE'] == 1]
train_y_3 = train_x['Y_Class'][train_x['PRODUCT_CODE'] == 2]

# test
test_x_1 = test_x[test_x['PRODUCT_CODE'] == 0]
test_x_2 = test_x[test_x['PRODUCT_CODE'] == 1]
test_x_3 = test_x[test_x['PRODUCT_CODE'] == 2]



# LINE

## TRAIN

# line 1
test_x_1_1 = test_x_1[test_x_1['LINE'] == 0]

# line 2
test_x_1_2 = test_x_1[test_x_1['LINE'] == 1]

# line 3
test_x_1_3 = test_x_1[test_x_1['LINE'] == 2]

# line 4
test_x_1_4 = test_x_1[test_x_1['LINE'] == 3]

# line 5
test_x_2_5 = test_x_2[test_x_2['LINE'] == 4]
test_x_3_5 = test_x_3[test_x_3['LINE'] == 4]

# line 6
test_x_2_6 = test_x_2[test_x_2['LINE'] == 5]
test_x_3_6 = test_x_3[test_x_3['LINE'] == 5]




## TRAIN

# line 1
train_x_1_1 = train_x_1[train_x_1['LINE'] == 0]

# line 2
train_x_1_2 = train_x_1[train_x_1['LINE'] == 1]

# line 3
train_x_1_3 = train_x_1[train_x_1['LINE'] == 2]

# line 4
train_x_1_4 = train_x_1[train_x_1['LINE'] == 3]

# line 5
train_x_2_5 = train_x_2[train_x_2['LINE'] == 4]
train_x_3_5 = train_x_3[train_x_3['LINE'] == 4]

# line 6
train_x_2_6 = train_x_2[train_x_2['LINE'] == 5]
train_x_3_6 = train_x_3[train_x_3['LINE'] == 5]


train_set = [train_x_1_1, train_x_1_2, train_x_1_3, train_x_1_4, train_x_2_5, train_x_3_5, train_x_2_6, train_x_3_6]
test_set = [test_x_1_1, test_x_1_2, test_x_1_3, test_x_1_4, test_x_2_5, test_x_3_5, test_x_2_6, test_x_3_6]

print('done')

done


In [120]:
train_set_mean = train_set
test_set_mean = test_set

for set in train_set_mean:
    cols = set.columns
    for col in cols:
        set[col] = set[col].fillna(set[col].mean())

for set in test_set_mean:
    cols = set.columns
    for col in cols:
        set[col] = set[col].fillna(set[col].mean())

train_x = pd.concat(train_set_mean, axis=0).sort_index()
test_x = pd.concat(test_set_mean, axis=0).sort_index()

In [121]:
train_x['LINE_PRODUCT_CODE'] = train_x[['LINE','PRODUCT_CODE']].apply(lambda x: '-'.join(x.astype(str)),axis=1)
test_x['LINE_PRODUCT_CODE'] = test_x[['LINE','PRODUCT_CODE']].apply(lambda x: '-'.join(x.astype(str)),axis=1)

train_x.drop(['LINE','PRODUCT_CODE'], axis=1, inplace=True)
test_x.drop(['LINE','PRODUCT_CODE'], axis=1, inplace=True)

le = LabelEncoder()
train_x['LINE_PRODUCT_CODE'] = le.fit_transform(train_x['LINE_PRODUCT_CODE'])
test_x['LINE_PRODUCT_CODE'] = le.transform(test_x['LINE_PRODUCT_CODE'])

In [122]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(0, 1))
train_x_s = scaler.fit_transform(train_x.drop('LINE_PRODUCT_CODE', axis=1))
train_x_s = pd.DataFrame(train_x_s)


test_x_s = scaler.transform(test_x.drop('LINE_PRODUCT_CODE', axis=1))
test_x_s = pd.DataFrame(test_x_s)

In [123]:
train_x_s['LINE_PRODUCT_CODE'] = train_x['LINE_PRODUCT_CODE']
test_x_s['LINE_PRODUCT_CODE'] = test_x['LINE_PRODUCT_CODE']

In [124]:
train_x = train_x_s.fillna(0)
test_x = test_x_s.fillna(0)

In [125]:
train_x['Y_Class'] = train_y

train_df = train_x

## 목표 변수 만들기

원래 데이터세트의 작업은 애완 동물이 입양되는 속도를 예측하는 것입니다(예: 첫번째 주, 첫번째 달, 처음 3개월 등). 이 튜토리얼에서는 이해를 돕기 위해 단순화해보겠습니다. 여기에서는 데이터세트를 이진 분류 문제로 변환하고 애완 동물이 입양되었는지 여부를 단순하게 예측합니다.

레이블 열을 수정한 후, 0은 애완 동물이 입양되지 않았음을 나타내고 1은 입양되었음을 나타냅니다.

In [29]:
'''# In the original dataset "4" indicates the pet was not adopted.
dataframe['target'] = np.where(dataframe['AdoptionSpeed']==4, 0, 1)

# Drop un-used columns.
dataframe = dataframe.drop(columns=['AdoptionSpeed', 'Description'])'''

'# In the original dataset "4" indicates the pet was not adopted.\ndataframe[\'target\'] = np.where(dataframe[\'AdoptionSpeed\']==4, 0, 1)\n\n# Drop un-used columns.\ndataframe = dataframe.drop(columns=[\'AdoptionSpeed\', \'Description\'])'

## 데이터프레임을 훈련 세트, 검증 세트, 테스트 세트로 나누기

하나의 CSV 파일에서 데이터셋을 다운로드했습니다. 이를 훈련 세트, 검증 세트, 테스트 세트로 나누겠습니다.

In [30]:
'''from sklearn.model_selection import train_test_split

x_t, x_v, y_t, y_v = train_test_split(train_df.drop('Y_Class', axis=1), train_df['Y_Class'], test_size=0.2, random_state=42)'''

"from sklearn.model_selection import train_test_split\n\nx_t, x_v, y_t, y_v = train_test_split(train_df.drop('Y_Class', axis=1), train_df['Y_Class'], test_size=0.2, random_state=42)"

In [126]:
train, val = train_test_split(train_df, test_size=0.15)
#train, val = train_test_split(train, test_size=0.15)
test = test_x

print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

508 train examples
90 validation examples
310 test examples


## tf.data를 사용하여 입력 파이프라인 만들기

그다음 [tf.data](https://www.tensorflow.org/guide/datasets)를 사용하여 데이터프레임을 감싸겠습니다. 이렇게 하면 특성 열을 사용하여 판다스 데이터프레임의 열을 모델 훈련에 필요한 특성으로 매핑할 수 있습니다. 아주 큰 CSV 파일(메모리에 들어갈 수 없을 정도로 큰 파일)을 다룬다면 tf.data로 디스크 디렉토리에서 데이터를 읽을 수 있습니다. 이런 내용은 이 튜토리얼에 포함되어 있지 않습니다.

In [85]:
# A utility method to create a tf.data dataset from a Pandas Dataframe
def df_to_dataset(train_df, shuffle=True, batch_size=10):
  train_df = train_df.copy()
  labels = train_df.pop('Y_Class')
  ds = tf.data.Dataset.from_tensor_slices((dict(train_df), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(train_df))
  ds = ds.batch(batch_size)
  return ds

In [33]:
'''# X
def df_to_dataset(train_df, shuffle=True, batch_size=10):
  train_df = train_df.copy()
  ds = tf.data.Dataset.from_tensor_slices((dict(train_df)))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(train_df))
  ds = ds.batch(batch_size)
  return ds


# Y
def y_to_dataset(train_df, shuffle=True, batch_size=10):
  train_df = train_df.copy()
  ds = tf.data.Dataset.from_tensor_slices((train_df))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(train_df))
  ds = ds.batch(batch_size)
  return ds

x_t_ds = df_to_dataset(x_t, batch_size=batch_size)
y_t_ds = y_to_dataset(y_t, batch_size=batch_size)
x_v_ds = df_to_dataset(x_v, batch_size=batch_size)
y_v_ds = y_to_dataset(y_v, batch_size=batch_size)'''

'# X\ndef df_to_dataset(train_df, shuffle=True, batch_size=10):\n  train_df = train_df.copy()\n  ds = tf.data.Dataset.from_tensor_slices((dict(train_df)))\n  if shuffle:\n    ds = ds.shuffle(buffer_size=len(train_df))\n  ds = ds.batch(batch_size)\n  return ds\n\n\n# Y\ndef y_to_dataset(train_df, shuffle=True, batch_size=10):\n  train_df = train_df.copy()\n  ds = tf.data.Dataset.from_tensor_slices((train_df))\n  if shuffle:\n    ds = ds.shuffle(buffer_size=len(train_df))\n  ds = ds.batch(batch_size)\n  return ds\n\nx_t_ds = df_to_dataset(x_t, batch_size=batch_size)\ny_t_ds = y_to_dataset(y_t, batch_size=batch_size)\nx_v_ds = df_to_dataset(x_v, batch_size=batch_size)\ny_v_ds = y_to_dataset(y_v, batch_size=batch_size)'

In [34]:
# 보류

'''batch_size = 5 # A small batch sized is used for demonstration purposes
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
#test_ds = df_to_dataset(test_df, shuffle=False, batch_size=batch_size)'''

'batch_size = 5 # A small batch sized is used for demonstration purposes\ntrain_ds = df_to_dataset(train, batch_size=batch_size)\nval_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)\n#test_ds = df_to_dataset(test_df, shuffle=False, batch_size=batch_size)'

## 사용할 열 선택하기

여러 가지 특성 열을 사용하는 방법을 보았으므로 이제 이를 사용하여 모델을 훈련하겠습니다. 이 튜토리얼의 목적은 특성 열을 사용하는 완전한 코드(예를 들면 작동 방식)를 제시하는 것이므로 임의로 몇 개의 열을 선택하여 모델을 훈련하겠습니다.

핵심 포인트: 제대로 된 모델을 만들어야 한다면 대용량의 데이터셋을 사용하고 어떤 특성을 포함하는 것이 가장 의미있는지, 또 어떻게 표현해야 할지 신중하게 생각하세요.

In [143]:
cols = train_df.columns.drop(['Y_Class', 'LINE_PRODUCT_CODE'])

new_cols = []

for col in cols:
    print(col)
    col = 'X_'+str(col+1)
    new_cols.append(col)
    print(col)


0
X_1
1
X_2
2
X_3
3
X_4
4
X_5
5
X_6
6
X_7
7
X_8
8
X_9
9
X_10
10
X_11
11
X_12
12
X_13
13
X_14
14
X_15
15
X_16
16
X_17
17
X_18
18
X_19
19
X_20
20
X_21
21
X_22
22
X_23
23
X_24
24
X_25
25
X_26
26
X_27
27
X_28
28
X_29
29
X_30
30
X_31
31
X_32
32
X_33
33
X_34
34
X_35
35
X_36
36
X_37
37
X_38
38
X_39
39
X_40
40
X_41
41
X_42
42
X_43
43
X_44
44
X_45
45
X_46
46
X_47
47
X_48
48
X_49
49
X_50
50
X_51
51
X_52
52
X_53
53
X_54
54
X_55
55
X_56
56
X_57
57
X_58
58
X_59
59
X_60
60
X_61
61
X_62
62
X_63
63
X_64
64
X_65
65
X_66
66
X_67
67
X_68
68
X_69
69
X_70
70
X_71
71
X_72
72
X_73
73
X_74
74
X_75
75
X_76
76
X_77
77
X_78
78
X_79
79
X_80
80
X_81
81
X_82
82
X_83
83
X_84
84
X_85
85
X_86
86
X_87
87
X_88
88
X_89
89
X_90
90
X_91
91
X_92
92
X_93
93
X_94
94
X_95
95
X_96
96
X_97
97
X_98
98
X_99
99
X_100
100
X_101
101
X_102
102
X_103
103
X_104
104
X_105
105
X_106
106
X_107
107
X_108
108
X_109
109
X_110
110
X_111
111
X_112
112
X_113
113
X_114
114
X_115
115
X_116
116
X_117
117
X_118
118
X_119
119
X_120
120
X_121
121
X_12

In [145]:
from tensorflow import feature_column


feature_columns = []

# numeric cols
for header in new_cols:
  feature_columns.append(feature_column.numeric_column(header))




# indicator_columns(category)
indicator_column_names = ['LINE_PRODUCT_CODE']
for col_name in indicator_column_names:
  categorical_column = feature_column.categorical_column_with_vocabulary_list(
      col_name, train_df[col_name].unique())
  indicator_column = feature_column.indicator_column(categorical_column)
  feature_columns.append(indicator_column)

In [36]:
'''# bucketized cols
age = feature_column.numeric_column('Age')
age_buckets = feature_column.bucketized_column(age, boundaries=[1, 2, 3, 4, 5])
feature_columns.append(age_buckets)'''

"# bucketized cols\nage = feature_column.numeric_column('Age')\nage_buckets = feature_column.bucketized_column(age, boundaries=[1, 2, 3, 4, 5])\nfeature_columns.append(age_buckets)"

In [38]:
'''# embedding columns
breed1 = feature_column.categorical_column_with_vocabulary_list(
      'Breed1', dataframe.Breed1.unique())
breed1_embedding = feature_column.embedding_column(breed1, dimension=8)
feature_columns.append(breed1_embedding)'''

"# embedding columns\nbreed1 = feature_column.categorical_column_with_vocabulary_list(\n      'Breed1', dataframe.Breed1.unique())\nbreed1_embedding = feature_column.embedding_column(breed1, dimension=8)\nfeature_columns.append(breed1_embedding)"

In [39]:
'''# crossed columns
age_type_feature = feature_column.crossed_column([age_buckets, animal_type], hash_bucket_size=100)
feature_columns.append(feature_column.indicator_column(age_type_feature))'''

'# crossed columns\nage_type_feature = feature_column.crossed_column([age_buckets, animal_type], hash_bucket_size=100)\nfeature_columns.append(feature_column.indicator_column(age_type_feature))'

### 특성 층 만들기

특성 열을 정의하고 나면 [DenseFeatures](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/layers/DenseFeatures) 층을 사용해 케라스 모델에 주입할 수 있습니다.

In [146]:
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)

앞서 특성 열의 작동 예를 보이기 위해 작은 배치 크기를 사용했습니다. 여기에서는 조금 더 큰 배치 크기로 입력 파이프라인을 만듭니다.

In [147]:
batch_size = 32    
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

TypeError: '<' not supported between instances of 'str' and 'int'

In [42]:
print(type(train_ds))
print(type(val_ds))

<class 'tensorflow.python.data.ops.dataset_ops.BatchDataset'>
<class 'tensorflow.python.data.ops.dataset_ops.BatchDataset'>


## 모델 생성, 컴파일, 훈련

In [43]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

In [44]:
model = tf.keras.Sequential([
  feature_layer,
  layers.Dense(128, activation='relu'),
  layers.Dense(128, activation='relu'),
  layers.Dropout(.1),
  layers.Dense(1)
])

In [46]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
'''history = model.fit(x_t_ds, y_t, epochs=40, 
                    validation_data=(x_v_ds, y_v))'''

In [48]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
history = model.fit(train_ds,
                    validation_data=val_ds,
                    epochs=100
                    )

In [ ]:
loss, accuracy = model.evaluate(test_ds)
print("Accuracy", accuracy)

3/3 [==============================] - 1s 348ms/step - loss: 7.7121e-08 - accuracy: 0.1556
Accuracy 0.15555556118488312


핵심 포인트: 일반적으로 크고 복잡한 데이터셋일 경우 딥러닝 모델에서 최선의 결과를 얻습니다. 이런 작은 데이터셋에서는 기본 모델로 결정 트리(decision tree)나 랜덤 포레스트(random forest)를 사용하는 것이 권장됩니다. 이 튜토리얼의 목적은 정확한 모델을 훈련하는 것이 아니라 정형 데이터를 다루는 방식을 설명하는 것입니다. 실전 데이터셋을 다룰 때 이 코드를 시작점으로 사용하세요.

## 그 다음엔

정형 데이터를 사용한 분류 작업에 대해 배우는 가장 좋은 방법은 직접 실습하는 것입니다. 실험해 볼 다른 데이터셋을 찾아서 위와 비슷한 코드를 사용해 모델을 훈련해 보세요. 정확도를 향상시키려면 모델에 포함할 특성과 표현 방법을 신중하게 생각하세요.